In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def splitter(x):
    row = np.array(x.split(',')).astype(np.float32)
    label = row[0]
    pixel = row[1:]
    return label, pixel

In [3]:
def normalize(label, pixel):
    """Returns a normalized feature array between -1 and 1"""
    return label, (pixel - np.min(pixel))/(np.max(pixel) - np.min(pixel)) 

In [4]:
trainpath = './fashionmnist/training.csv'
validationpath = './fashionmnist/validation.csv'

In [5]:
training_dataset = tf.data.TextLineDataset(trainpath).skip(1).filter(lambda line: tf.not_equal(tf.substr(line, 0, 1), "#"))
validation_dataset = tf.data.TextLineDataset(validationpath).skip(1).filter(lambda line: tf.not_equal(tf.substr(line, 0, 1), "#"))

In [6]:
training_dataset = training_dataset.map(lambda x: tf.py_func(splitter, [x], [tf.float32, tf.float32]))
validation_dataset = validation_dataset.map(lambda x: tf.py_func(splitter, [x], [tf.float32, tf.float32]))

In [7]:
training_dataset = training_dataset.map(lambda label, pixel: tf.py_func(normalize, [label, pixel], [tf.float32, tf.float32]))

In [8]:
training_dataset = training_dataset.shuffle(buffer_size=10000)

In [9]:
training_dataset = training_dataset.batch(100)
validation_dataset = validation_dataset.batch(100)

In [10]:
# A feedable iterator is defined by a handle placeholder and its structure. We
# could use the `output_types` and `output_shapes` properties of either
# `training_dataset` or `validation_dataset` here, because they have
# identical structure.
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

In [11]:
training_dataset.output_shapes

(TensorShape(None), TensorShape(None))

In [12]:
# You can use feedable iterators with a variety of different kinds of iterator
# (such as one-shot and initializable iterators).
training_iterator = training_dataset.make_initializable_iterator()
validation_iterator = validation_dataset.make_initializable_iterator()

In [13]:
with tf.Session() as sess:
    # The `Iterator.string_handle()` method returns a tensor that can be evaluated
    # and used to feed the `handle` placeholder.
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())
    
    for i in range(1):
        sess.run(training_iterator.initializer)
        while True:
            try:
                label_batch, image_batch = sess.run(next_element, feed_dict={handle: training_handle})
                print(image_batch)
            except tf.errors.OutOfRangeError:
                print('Out of Data, Training Finished!')
            finally:
                sess.run(validation_iterator.initializer)
                sess.run(next_element, feed_dict={handle: validation_handle})
        if i%10==0:
            sess.run(validation_iterator.initializer)
            sess.run(next_element, feed_dict={handle: validation_handle})

[[ 0.          0.          0.         ...,  0.          0.          0.03529412]
 [ 0.          0.          0.         ...,  0.          0.          0.01568628]
 [ 0.          0.          0.         ...,  0.          0.          0.01176471]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.02745098]
 [ 0.          0.          0.         ...,  0.          0.          0.03529412]
 [ 0.          0.          0.         ...,  0.          0.          0.02755906]]
[[ 0.          0.          0.         ...,  0.          0.          0.03137255]
 [ 0.          0.          0.         ...,  0.          0.          0.02352941]
 [ 0.          0.          0.         ...,  0.          0.          0.02745098]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.00784314]
 [ 0.          0.          0.         ...,  0.          0.          0.02352941]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]
[[ 0.          0.       

KeyboardInterrupt: 